In [1]:
#CS221_11292018_0824 includes a RAKE class that applies RAKE on the input file
import tensorflow as tf
import pandas as pd
import json
import numpy as np
import sys
import collections
import gzip
import statistics as stat
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize 
from nltk.stem import WordNetLemmatizer

In [2]:
#reading data
        
#This block reads 5-core dataset
# number_of_data_points_to_read= 10000 #Select number of data points to read
counter= 0
data=[]
path= '/Users/srishti/Desktop/reviews_Health_and_Personal_Care_5.json'
with open(path) as f:
    for line in f:
#         if counter <number_of_data_points_to_read:
        data.append(json.loads(line))
        counter+=1
#         else: 
#             break

X=[] #reivews
Y=[] #ratings



In [60]:
asin_mother_dict= collections.defaultdict(dict)
asin_pos_reviews_dict = collections.defaultdict(list)
temp_dict_all_parameters_known={}
# counter= len(data)
counter=0
#Number of data_points_to_select
number_of_data_points_to_select= len(data)
for i in data:
    if counter < number_of_data_points_to_select:
        counter+=1
        if i['asin'] not in asin_mother_dict.keys():
            temp_dict={}
            temp_dict['pos_reviews']=""
            temp_dict['neg_reviews']=""
            asin_mother_dict[i['asin']]=temp_dict
            asin_pos_reviews_dict[i['asin']]= "" 

    #     asin_pos_reviews_dict[i['asin']].append(i['reviewText'])
        if i['overall'] >= 4:
            temp_dict={}
            temp_dict['pos_reviews']= asin_mother_dict[i['asin']]['pos_reviews']+ " "+ i['reviewText']
            asin_mother_dict[i['asin']].update(temp_dict) 
            asin_pos_reviews_dict[i['asin']]= asin_mother_dict[i['asin']]

        else:
            temp_dict={}
            temp_dict['neg_reviews']= asin_mother_dict[i['asin']]['neg_reviews']+ " "+ i['reviewText']
            asin_mother_dict[i['asin']].update(temp_dict) 
    #         asin_mother_dict[i['asin']]['neg_reviews'] = asin_mother_dict[i['asin']]['neg_reviews']+ " "+ i['reviewText']

    for i in asin_mother_dict.keys():
        if asin_mother_dict[i]['pos_reviews']=="":
            asin_mother_dict[i]['all_param_known']='False'
        else:
            asin_mother_dict[i]['all_param_known']='True'



In [61]:
counter=0
for i in asin_mother_dict.keys():
    if counter <2:
        print i+ " "+ str(asin_mother_dict[i])
        counter+=1
        

3812028492 {'all_param_known': 'True', 'neg_reviews': u' Real nice build to this razor. It\'s hard to beat the German quality. However, I believe this razor is not aggressive enough. I actually got a  better shave with my $12.00 Lord. I just don\'t get that close shave feeling, even after two passes. Maybe I should try a third pass, but I don\'t think it\'s going to matter. I\'ve used both Feather and the Gillette 7 O\'clock Sharp Edge blades. The new feather worked better this morning. I\'ll try it for a few days more and see how it goes. I had this razor for 5 years.  Finally, the handle separated from the head of the razor while closing the razor head with the knob at the bottom.  There is a screw head that comes out of the handle, which the razor head attaches by surrounding the screw head with with plating.  I wasn\'t abusing the razor or over-tightening it.  I was hoping to get more "bang for the buck" out of this razor.  I think the longer handle of this razor enables more force

In [14]:
# asin_mother_dict_copy= asin_mother_dict.copy()


In [12]:
# asin_mother_dict= asin_mother_dict_copy.copy()


In [15]:
#This block reads meta data
#reading meta data
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield eval(l)

meta_data = []
# meta_path= '/Users/srishti/Desktop/meta_Books.json.gz'
meta_path= '/Users/srishti/Desktop/meta_Health_and_Personal_Care.json.gz'
counter= 1


for line in parse(meta_path):
    meta_data.append(line)

In [62]:

asin_title_map = collections.defaultdict(list)
all_asins_set= set(asin_mother_dict.keys())

for i in meta_data:
    temp_dict={}
    temp_dict_all_parameters_known={}
    if i['asin'] in all_asins_set:
        all_asins_set.remove(i['asin'])
        #Getting Titles
        if 'title' in i.keys():
            temp_dict['title']= i['title']
#             print i['asin']
#             print asin_mother_dict[i['asin']]
            asin_mother_dict[i['asin']].update(temp_dict)
    #         asin_title_map[i['asin']] = i['title']

        else:
            temp_dict['title']= None
            temp_dict['all_param_known']='False'
            asin_mother_dict[i['asin']].update(temp_dict)
            asin_title_map[i['asin']] = None

        #Adding also_bought, also_viewed, bought_together and buy_after_viewing
        temp_dict={}
        temp_dict_all_parameters_known={}
        if 'related' in i.keys():
            if 'also_bought' in i['related']:
                temp_dict['also_bought']= i['related']['also_bought']
            else:
                temp_dict['also_bought']= None

            if 'also_viewed' in i['related']:
                temp_dict['also_viewed']= i['related']['also_viewed']
            else:
                temp_dict['also_viewed']= None

            if 'bought_together' in i['related']:
                temp_dict['bought_together']= i['related']['bought_together']
            else:
                temp_dict['bought_together']= None

            if 'buy_after_viewing' in i['related']:
                temp_dict['buy_after_viewing']= i['related']['buy_after_viewing']
            else:
                temp_dict['buy_after_viewing']= None

            asin_mother_dict[i['asin']].update(temp_dict)
        else:
            temp_dict['all_param_known']='False'
            temp_dict['also_viewed']= None
            temp_dict['also_bought']= None
            temp_dict['bought_together']= None
            temp_dict['buy_after_viewing']= None

            asin_mother_dict[i['asin']].update(temp_dict)

        #Adding categories
        #NOTE HEALTH & PERSONAL CARE IS A CATEGORY FOR ALL PRODUCT, NEED TO ADDRESS THAT
        temp_dict={}
        temp_dict_all_parameters_known={}
        if 'categories' in i.keys():
            temp_dict['categories']= i['categories'][0]
            asin_mother_dict[i['asin']].update(temp_dict)
        else:
            temp_dict['categories']= None
            temp_dict['all_param_known']='False'
            asin_mother_dict[i['asin']].update(temp_dict)


            

In [63]:
count=0
for i in asin_mother_dict.keys():
    if count < 2:
        print i
        print asin_mother_dict[i]
        count+=1

3812028492
{'buy_after_viewing': None, 'title': 'Merkur Long Handled Safety Razor 38c', 'pos_reviews': u" great quality, feels very good in my hand, very good weight, non slippery handle. it's a very good buy, cant wrong with this one. 34c is also very good, but shorter handle. Make no mistake this is a sturdy long handled razor. Feels substantial in your hand and the finish makes it easy to hold onto when wet. Easy to keep clean with the two piece design. Plating was beautiful and very good quality razor. My first safety razor, and I'll probably never have to buy another.My list of Amazon-acquired shaving supplies:Deluxe Stainless Steel Safety Razor and Shaving Brush Stand from Super Safety RazorsMerkur Classic Barbor Pole Long Handle Safety Razor #38 + 10 Free DE Razor BladesTruefitt & Hill Ultimate Comfort Pre-Shave Oil, 2 oz.Truefitt & Hill 1805 Aftershave BalmTweezerman  Men's Shaving BrushTruefitt & Hill 1805 Shave Cream JarOsma Styptic Pencil, Hemo StopFeather Hi-Stainless Plati

In [19]:
asin_mother_dict_copy= asin_mother_dict.copy()

In [64]:
for i in asin_mother_dict.keys():
    if asin_mother_dict[i]['all_param_known']!= 'True':
        del(asin_mother_dict[i])
print len(asin_mother_dict)

466


In [21]:
#
embeddings_index = dict()
f = open('/Users/srishti/Google Drive/000_7th Quarter/CS221/Project/untouched_data/glove.6B/glove.6B.50d.txt')
for line in f:# first element of each line is the word and remaining elements are numerical represenation of each line 
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs# creates a dict of words and numerical representation of that word
f.close()



In [22]:
embeddings_index['dog']
embeddings_index['time']

array([ 0.02648  ,  0.33737  ,  0.065667 , -0.11609  ,  0.41651  ,
       -0.21142  , -0.69582  ,  0.2822   , -0.36077  , -0.13822  ,
        0.012094 ,  0.086227 , -0.84638  ,  0.057195 ,  1.1582   ,
        0.14703  , -0.0049197, -0.24899  , -0.96014  , -0.3038   ,
        0.23972  ,  0.21058  ,  0.40608  ,  0.17789  ,  0.55253  ,
       -1.6357   , -0.17784  , -0.45222  ,  0.45805  ,  0.14239  ,
        3.7087   ,  0.40289  , -0.4083   , -0.29304  ,  0.030857 ,
       -0.15361  ,  0.10607  ,  0.63397  ,  0.12397  , -0.25349  ,
       -0.10344  ,  0.0069768, -0.17328  ,  0.35536  , -0.46369  ,
        0.15285  ,  0.41475  , -0.3398   , -0.23043  ,  0.19069  ],
      dtype=float32)

In [65]:
# -*- coding: utf-8 -*-
"""Implementation of Rapid Automatic Keyword Extraction algorithm.
As described in the paper `Automatic keyword extraction from individual
documents` by Stuart Rose, Dave Engel, Nick Cramer and Wendy Cowley.
"""

import string
from collections import Counter, defaultdict
from itertools import chain, groupby, product

import nltk
from enum import Enum
from nltk.tokenize import wordpunct_tokenize


class Metric(Enum):
    """Different metrics that can be used for ranking."""

    DEGREE_TO_FREQUENCY_RATIO = 0  # Uses d(w)/f(w) as the metric
    WORD_DEGREE = 1  # Uses d(w) alone as the metric
    WORD_FREQUENCY = 2  # Uses f(w) alone as the metric


class Rake(object):
    """Rapid Automatic Keyword Extraction Algorithm."""

    def __init__(
        self,
        stopwords=None,
        punctuations=None,
        language="english",
        ranking_metric=Metric.DEGREE_TO_FREQUENCY_RATIO,
        max_length=10000,
        min_length=1,
    ):
        """Constructor.
        :param stopwords: List of Words to be ignored for keyword extraction.
        :param punctuations: Punctuations to be ignored for keyword extraction.
        :param language: Language to be used for stopwords
        :param max_length: Maximum limit on the number of words in a phrase
                           (Inclusive. Defaults to 100000)
        :param min_length: Minimum limit on the number of words in a phrase
                           (Inclusive. Defaults to 1)
        """
        # By default use degree to frequency ratio as the metric.
        if isinstance(ranking_metric, Metric):
            self.metric = ranking_metric
        else:
            self.metric = Metric.DEGREE_TO_FREQUENCY_RATIO

        # If stopwords not provided we use language stopwords by default.
        self.stopwords = stopwords
        if self.stopwords is None:
            self.stopwords = nltk.corpus.stopwords.words(language)

        # If punctuations are not provided we ignore all punctuation symbols.
        self.punctuations = punctuations
        if self.punctuations is None:
            self.punctuations = string.punctuation

        # All things which act as sentence breaks during keyword extraction.
        self.to_ignore = set(chain(self.stopwords, self.punctuations))

        # Assign min or max length to the attributes
        self.min_length = min_length
        self.max_length = max_length

        # Stuff to be extracted from the provided text.
        self.frequency_dist = None
        self.degree = None
        self.rank_list = None
        self.ranked_phrases = None

    def extract_keywords_from_text(self, text):
        """Method to extract keywords from the text provided.
        :param text: Text to extract keywords from, provided as a string.
        """
        sentences = nltk.tokenize.sent_tokenize(text)
        self.extract_keywords_from_sentences(sentences)

    def extract_keywords_from_sentences(self, sentences):
        """Method to extract keywords from the list of sentences provided.
        :param sentences: Text to extraxt keywords from, provided as a list
                          of strings, where each string is a sentence.
        """
        phrase_list = self._generate_phrases(sentences)
        self._build_frequency_dist(phrase_list)
        self._build_word_co_occurance_graph(phrase_list)
        self._build_ranklist(phrase_list)

    def get_ranked_phrases(self):
        """Method to fetch ranked keyword strings.
        :return: List of strings where each string represents an extracted
                 keyword string.
        """
        return self.ranked_phrases

    def get_ranked_phrases_with_scores(self):
        """Method to fetch ranked keyword strings along with their scores.
        :return: List of tuples where each tuple is formed of an extracted
                 keyword string and its score. Ex: (5.68, 'Four Scoures')
        """
        return self.rank_list

    def get_word_frequency_distribution(self):
        """Method to fetch the word frequency distribution in the given text.
        :return: Dictionary (defaultdict) of the format `word -> frequency`.
        """
        return self.frequency_dist

    def get_word_degrees(self):
        """Method to fetch the degree of words in the given text. Degree can be
        defined as sum of co-occurances of the word with other words in the
        given text.
        :return: Dictionary (defaultdict) of the format `word -> degree`.
        """
        return self.degree

    def _build_frequency_dist(self, phrase_list):
        """Builds frequency distribution of the words in the given body of text.
        :param phrase_list: List of List of strings where each sublist is a
                            collection of words which form a contender phrase.
        """
        self.frequency_dist = Counter(chain.from_iterable(phrase_list))

    def _build_word_co_occurance_graph(self, phrase_list):
        """Builds the co-occurance graph of words in the given body of text to
        compute degree of each word.
        :param phrase_list: List of List of strings where each sublist is a
                            collection of words which form a contender phrase.
        """
        co_occurance_graph = defaultdict(lambda: defaultdict(lambda: 0))
        for phrase in phrase_list:
            # For each phrase in the phrase list, count co-occurances of the
            # word with other words in the phrase.
            #
            # Note: Keep the co-occurances graph as is, to help facilitate its
            # use in other creative ways if required later.
            for (word, coword) in product(phrase, phrase):
                co_occurance_graph[word][coword] += 1
        self.degree = defaultdict(lambda: 0)
        for key in co_occurance_graph:
            self.degree[key] = sum(co_occurance_graph[key].values())

    def _build_ranklist(self, phrase_list):
        """Method to rank each contender phrase using the formula
              phrase_score = sum of scores of words in the phrase.
              word_score = d(w)/f(w) where d is degree and f is frequency.
        :param phrase_list: List of List of strings where each sublist is a
                            collection of words which form a contender phrase.
        """
        self.rank_list = []
        for phrase in phrase_list:
            rank = 0.0
            for word in phrase:
                if self.metric == Metric.DEGREE_TO_FREQUENCY_RATIO:
                    rank += 1.0 * self.degree[word] / self.frequency_dist[word]
                elif self.metric == Metric.WORD_DEGREE:
                    rank += 1.0 * self.degree[word]
                else:
                    rank += 1.0 * self.frequency_dist[word]
            self.rank_list.append((rank, " ".join(phrase)))
        self.rank_list.sort(reverse=True)
        self.ranked_phrases = [ph[1] for ph in self.rank_list]

    def _generate_phrases(self, sentences):
        """Method to generate contender phrases given the sentences of the text
        document.
        :param sentences: List of strings where each string represents a
                          sentence which forms the text.
        :return: Set of string tuples where each tuple is a collection
                 of words forming a contender phrase.
        """
        phrase_list = set()
        # Create contender phrases from sentences.
        for sentence in sentences:
            word_list = [word.lower() for word in wordpunct_tokenize(sentence)]
            phrase_list.update(self._get_phrase_list_from_words(word_list))
        return phrase_list

    def _get_phrase_list_from_words(self, word_list):
        """Method to create contender phrases from the list of words that form
        a sentence by dropping stopwords and punctuations and grouping the left
        words into phrases. Only phrases in the given length range (both limits
        inclusive) would be considered to build co-occurrence matrix. Ex:
        Sentence: Red apples, are good in flavour.
        List of words: ['red', 'apples', ",", 'are', 'good', 'in', 'flavour']
        List after dropping punctuations and stopwords.
        List of words: ['red', 'apples', *, *, good, *, 'flavour']
        List of phrases: [('red', 'apples'), ('good',), ('flavour',)]
        List of phrases with a correct length:
        For the range [1, 2]: [('red', 'apples'), ('good',), ('flavour',)]
        For the range [1, 1]: [('good',), ('flavour',)]
        For the range [2, 2]: [('red', 'apples')]
        :param word_list: List of words which form a sentence when joined in
                          the same order.
        :return: List of contender phrases that are formed after dropping
                 stopwords and punctuations.
        """
        groups = groupby(word_list, lambda x: x not in self.to_ignore)
        phrases = [tuple(group[1]) for group in groups if group[0]]
        return list(
            filter(
                lambda x: self.min_length <= len(x) <= self.max_length, phrases
            )
         )

In [66]:
###########################
# Helper functions
###########################

# Generate frequency dictionary of all words in all reviews (needed for TF-IDF)
def generateCorpus(X):
    all_words = defaultdict(int)
    for k in range(0,len(X)):
		wordList = re.sub("[^\w]", " ",  X[k]).split() # Clean and split data
		for words in wordList:
			all_words[words.lower()] += 1
    
    return all_words

class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t, 'n') for t in word_tokenize(doc)]

def TF_IDF_all(asin_dict):

    vectorizer = TfidfVectorizer(stop_words="english", analyzer='word', lowercase = True, tokenizer = LemmaTokenizer(), 
				 ngram_range=(1, 2), min_df = 2, max_df = 0.8)

    vec = vectorizer.fit_transform(list(asin_dict.values()))
    word_map=vectorizer.get_feature_names()

    return vec, word_map

def TF_IDF_asin(asin, asin_dict, vectorizer, word_map, n):
    index = asin_dict.keys().index(asin)
    tf = vectorizer[index]
    
    keywords = []
    for col in tf.nonzero()[1]:
        keywords.append((tf[0, col],word_map[col]))

    sorted_keywords = sorted(keywords, key=lambda t: t[1] * -1)

    return sorted_keywords[:n]


In [72]:
count=0
for i in asin_mother_dict.keys():
    if count< 2:
#         print asin_mother_dict[i]['pos_reviews']
        print type (asin_mother_dict[i])
#         print type (asin_mother_dict[i])
        count+=1
        

<type 'unicode'>
<type 'dict'>
<type 'unicode'>
<type 'dict'>


In [83]:
asin_dict= {}
for i in asin_mother_dict.keys():
#     print type(asin_mother_dict[i])
#     print (asin_mother_dict[i]['pos_reviews'])
    if asin_mother_dict[i]['all_param_known']=='True' :
        asin_dict[i]= asin_mother_dict[i]['pos_reviews']

number_of_asins_on_which_keyword_separators_should_be_applied= len(asin_dict.keys())
counter=0
rake_keywords_dict={}
TFIDF_keywords_dict={}
vec, map = TF_IDF_all(asin_dict)

for i in asin_dict.keys():
    if counter < number_of_asins_on_which_keyword_separators_should_be_applied:
#         print asin_dict[i]
        c = Rake()
        c.extract_keywords_from_text(asin_dict[i])
        rake_keywords_dict[i]= c.rank_list
        counter+=1
        
        ###########################
        # Run TF-IDF
        ###########################
#         #test_list = [0, 2, 3, 4, 8, 9, 12, 13, 14]
#         test_list = [0]

#         for i in test_list:
#             my_asin = asin_dict.keys()[i]
#             print my_asin
#         print asin_title_map[i]

        TFIDF_keywords_dict[i] = TF_IDF_asin(i, asin_dict, vec, map, 10)
#             print keywords
    else:
        break
        #     rl_list.append(c.rank_list)




In [103]:
# d = Rake()
# d.extract_keywords_from_text('I love cats and cats')
# print d.rank_list
# print d.ranked_phrases
print len(TFIDF_keywords_dict.keys())


100


In [104]:
#converts reviews to embedding dict where key is the ASIN of a book
count=0
RAKE_review_dict={}
TFIDF_review_dict={}

lower_cutoff_for_word_to_be_considered=1#model parameter
TFIDF_lower_cutoff_for_word_to_be_considered=0.0#model parameter

number_of_reviews_for_clustering=100#select how many reviews are being used for clustering, small number reduces computation time. 

max_word_per_review= 10#model parameter
TFIDF_max_word_per_review= 10#model parameter


if number_of_reviews_for_clustering > len(rake_keywords_dict.keys()):
    print ("number_of_reviews_for_clustering too large!!")
    print("number of reviews= " + str(len(rake_keywords_dict.keys()))+ " requested " + str(number_of_reviews_for_clustering))
else:
    for i in rake_keywords_dict.keys():
        normalization_factor=0
#         print i
        if count< number_of_reviews_for_clustering :
            print count
            count +=1
    #         print rake_keywords_dict[i]
            count_of_words_used=0
#             print "entering RAKE"
            for j in rake_keywords_dict[i]:
#                 print "entered RAKE"
                if count_of_words_used < max_word_per_review:
                    count_of_words_used+=1
                    if j[0] > lower_cutoff_for_word_to_be_considered:
                        if j[1] not in embeddings_index.keys():
                            for k in j[1].split():
                #                 print ("k=" + k)
                                if k in embeddings_index.keys():
                                    if i not in RAKE_review_dict.keys():

                                        RAKE_review_dict[i]= j[0]* embeddings_index[k] 
                                        normalization_factor+=1
                                    else:
                                        RAKE_review_dict[i]+= j[0]* embeddings_index[k] 
                                        normalization_factor+=1
        #                         else:
        #                             print k
                        else:
                            if j[1] in embeddings_index.keys():
                #                 print ("j=" + j[1])
                    #             RAKE_review_dict[i]+= rl_list[j][0]* embeddings_index[j][1]
                                if i not in RAKE_review_dict.keys():
                                    RAKE_review_dict[i]= j[0]* embeddings_index[j[1]]
                                    normalization_factor+=1
                                else:
                                    RAKE_review_dict[i]+= j[0]* embeddings_index[j[1]]
                                    normalization_factor+=1
        #                     else:
        #                         print j[1]
#             print ("entering normalization" + str(i))
            if normalization_factor >0:
#                     RAKE_review_dict[i]= RAKE_review_dict[i]/ normalization_factor
                [z/normalization_factor for z in RAKE_review_dict[i]]
#             print ("RAKE done..")

    #TF-IDF converts reviews to embedding dict where key is the ASIN of a book
#             print ("RAKE done..")
            TFIDF_normalization_factor=0
            TFIDF_count_of_words_used=0
#             print TFIDF_keywords_dict[i]
#             print "entering TFIDF"
            for j in TFIDF_keywords_dict[i]:
                
                if TFIDF_count_of_words_used < max_word_per_review:
                    TFIDF_count_of_words_used+=1
                    if j[0] > TFIDF_lower_cutoff_for_word_to_be_considered:
                        if j[1] not in embeddings_index.keys():
                            for k in j[1].split():
#                                 print ("k=" + k)
                                if k in embeddings_index.keys():
#                                     print ("k=" + k)
                                    if i not in TFIDF_review_dict.keys():
                                        TFIDF_review_dict[i]= j[0]* embeddings_index[k] 
                                        TFIDF_normalization_factor+=1
                                    else:
                                        TFIDF_review_dict[i]+= j[0]* embeddings_index[k] 
                                        TFIDF_normalization_factor+=1
        #                         else:
        #                             print k
                        else:
                            if j[1] in embeddings_index.keys():
#                                 print ("j=" + j[1])
                                if i not in TFIDF_review_dict.keys():
                                    TFIDF_review_dict[i]= j[0]* embeddings_index[j[1]]
                                    TFIDF_normalization_factor+=1
                                else:
                                    TFIDF_review_dict[i]+= j[0]* embeddings_index[j[1]]
                                    TFIDF_normalization_factor+=1
        #                     else:
        #                         print j[1]
#             print ("entering TFIDF Normalization" + str(i))
#             print TFIDF_review_dict
            if TFIDF_normalization_factor >0:
#                     RAKE_review_dict[i]= RAKE_review_dict[i]/ normalization_factor
                [z/TFIDF_normalization_factor for z in TFIDF_review_dict[i]]






0


/Users/srishti/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:33: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
/Users/srishti/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:36: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
/Users/srishti/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:75: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
/Users/srishti/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:89: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47


/Users/srishti/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:78: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71


/Users/srishti/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:47: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [105]:
print len(TFIDF_review_dict.keys())


100


In [106]:
count =0
for i in RAKE_review_dict.keys():
    if i not in TFIDF_review_dict.keys():
        print i
        

In [108]:
RAKE_review_dict_list=[]
TFIDF_review_dict_list=[]
for i in RAKE_review_dict.keys():
    RAKE_review_dict_list.append(RAKE_review_dict[i])
    TFIDF_review_dict_list.append(TFIDF_review_dict[i])

In [222]:
#holy K-means
from sklearn.cluster import KMeans
RAKE_kmeans = KMeans(n_clusters=10, random_state=0).fit_predict(RAKE_review_dict_list)
TFIDF_kmeans= KMeans(n_clusters=10, random_state=0).fit_predict(TFIDF_review_dict_list)
print RAKE_kmeans
print TFIDF_kmeans





[3 2 7 1 1 7 1 3 4 4 4 1 7 4 1 7 1 1 1 4 1 5 3 3 7 3 1 7 3 3 3 7 0 7 7 1 3
 7 1 1 3 7 3 7 3 9 3 1 6 7 3 1 5 7 6 7 3 1 1 1 7 7 5 4 7 1 7 7 3 1 1 1 1 9
 8 1 7 6 7 1 7 4 7 1 4 4 7 4 7 4 7 1 1 1 1 7 7 4 7 1]
[0 5 6 1 5 7 5 3 3 2 2 9 0 0 0 3 9 3 0 3 9 3 0 5 2 0 2 5 0 3 0 3 0 3 4 3 3
 3 5 3 3 9 9 0 0 3 0 5 0 0 5 5 0 3 0 0 0 6 5 3 3 3 0 0 3 1 6 0 3 3 8 3 3 0
 0 0 3 0 3 3 3 0 3 1 3 3 3 3 0 0 3 9 9 3 5 3 3 0 4 3]


In [225]:
#matches the embeddings of review to a title
RAKE_clusteredTitles= collections.defaultdict(list)
RAKE_clustered_asins= collections.defaultdict(list)
for i in range(0, len(RAKE_kmeans)):
    for asin_value, word2VecList in RAKE_review_dict.items():
        if (word2VecList == RAKE_review_dict_list[i]).all():
#             print asin_value
            RAKE_clusteredTitles[RAKE_kmeans[i]].append(asin_mother_dict[asin_value]['title'])
#             temp_dict={}
#             temp_dict['asins']= RAKE_clustered_asins[RAKE_kmeans[i]].append(asin_value)
            RAKE_clustered_asins[RAKE_kmeans[i]].append(asin_value)
            break
print ("RAKE_Clustered Titles")
# print clusteredTitles

TFIDF_clusteredTitles= collections.defaultdict(list)
TFIDF_clustered_asins= collections.defaultdict(list)
for i in range(0, len(TFIDF_kmeans)):
    for asin_value, word2VecList in TFIDF_review_dict.items():
        if (word2VecList == TFIDF_review_dict_list[i]).all():
#             print asin_value
            TFIDF_clusteredTitles[TFIDF_kmeans[i]].append(asin_mother_dict[asin_value]['title'])
            TFIDF_clustered_asins[TFIDF_kmeans[i]].append(asin_value)
            break
print ("TFIDF_Clustered Titles")
# print TFIDF_clusteredTitles




RAKE_Clustered Titles
TFIDF_Clustered Titles


In [226]:
for i in RAKE_clusteredTitles.keys():
    print "RAKE_Cluster Number: " + str(i)+ str(RAKE_clustered_asins[i])

for i in TFIDF_clusteredTitles.keys():
    print "TFIDF_Cluster Number: " + str(i)+ str(TFIDF_clustered_asins[i])
    


RAKE_Cluster Number: 0[u'B00008PW1R']
RAKE_Cluster Number: 1[u'B000052Y44', u'B0000537YG', u'B00009LI88', u'B0000C4KJ9', u'B000052YFO', u'B00008WFLP', u'B00008IJYS', u'B0000C4KJX', u'B000050B6Z', u'B000050B6B', u'B0000530YM', u'B00005NFBC', u'B00007MII0', u'B000052XKL', u'B00008O2XM', u'B000052YCG', u'B000050FE2', u'B00009ZOHD', u'B0000532OS', u'B000092YOX', u'B0000E3J9G', u'B0000E3J9F', u'B0000E3J9B', u'B00005335B', u'B000099SKD', u'B0000AN9LO', u'B0000AQOEW', u'B00002N88C', u'B000052YIY', u'B0000AU1ZU', u'B000087BHJ']
RAKE_Cluster Number: 2[u'3812028492']
RAKE_Cluster Number: 3[u'B000052Y5U', u'B00007GD5L', u'B0000DAZ2G', u'B000052YOH', u'B000052YOB', u'B0000DGBI2', u'B00008K4UA', u'B000053072', u'B000065CIL', u'B00007MII1', u'B00005BAWO', u'B00006IV1N', u'B0000DAPGS', u'B00005317T', u'B00005V3D9', u'B000052X6U']
RAKE_Cluster Number: 4[u'B000052YF7', u'B000052YHR', u'B000052YHS', u'B000052YFJ', u'B0000531E2', u'B00007KUX7', u'B00008LUTV', u'B000092YP2', u'B000092YP1', u'B000053136', 

In [244]:
def createOutputDicts(clustered_asins_dict):
#Create output dicts
    output_dict={}
    for i in clustered_asins_dict.keys():
        temp_dict={}
        temp_dict['asins']= clustered_asins_dict[i]
        output_dict[i]= temp_dict
    return output_dict
    # print RAKE_output_dict

# TFIDF_output_dict={}
# for i in TFIDF_clustered_asins.keys():
#     temp_dict={}
#     temp_dict['asins']= TFIDF_clustered_asins[i]
#     TFIDF_output_dict[i]= temp_dict
# print TFIDF_output_dict

In [245]:
RAKE_output_dict= createOutputDicts(RAKE_clustered_asins)
TFIDF_output_dict=createOutputDicts(TFIDF_clustered_asins)

In [246]:
print RAKE_output_dict
print TFIDF_output_dict

{0: {'asins': [u'B00008PW1R']}, 1: {'asins': [u'B000052Y44', u'B0000537YG', u'B00009LI88', u'B0000C4KJ9', u'B000052YFO', u'B00008WFLP', u'B00008IJYS', u'B0000C4KJX', u'B000050B6Z', u'B000050B6B', u'B0000530YM', u'B00005NFBC', u'B00007MII0', u'B000052XKL', u'B00008O2XM', u'B000052YCG', u'B000050FE2', u'B00009ZOHD', u'B0000532OS', u'B000092YOX', u'B0000E3J9G', u'B0000E3J9F', u'B0000E3J9B', u'B00005335B', u'B000099SKD', u'B0000AN9LO', u'B0000AQOEW', u'B00002N88C', u'B000052YIY', u'B0000AU1ZU', u'B000087BHJ']}, 2: {'asins': [u'3812028492']}, 3: {'asins': [u'B000052Y5U', u'B00007GD5L', u'B0000DAZ2G', u'B000052YOH', u'B000052YOB', u'B0000DGBI2', u'B00008K4UA', u'B000053072', u'B000065CIL', u'B00007MII1', u'B00005BAWO', u'B00006IV1N', u'B0000DAPGS', u'B00005317T', u'B00005V3D9', u'B000052X6U']}, 4: {'asins': [u'B000052YF7', u'B000052YHR', u'B000052YHS', u'B000052YFJ', u'B0000531E2', u'B00007KUX7', u'B00008LUTV', u'B000092YP2', u'B000092YP1', u'B000053136', u'B00006IV4N', u'B0000U1OCI']}, 5: {

In [254]:
RAKE_output_dict=updateOutputDict(RAKE_output_dict,asin_mother_dict)
TFIDF_output_dict= updateOutputDict(TFIDF_output_dict, asin_mother_dict)
# print RAKE_output_dict
# print TFIDF_output_dict


In [280]:
def findTopCategories(category_dict, number_of_categories_to_pick):
    list_top_counts=[]
    list_top_categories=[]
    dict_top={}
    number_of_dict_picked= number_of_categories_to_pick
    for i in category_dict.keys():
        if len(list_top_counts) < number_of_dict_picked:
            list_top_counts.append(category_dict[i])
            list_top_categories.append(i)
        else:
            min_value= min(list_top_counts)
            if  min_value < category_dict[i]:
                
#                 print i +" " + str(category_dict[i])
#                 print list_top_four_categories
                list_top_counts[list_top_counts.index(min_value)]= category_dict[i]
                for j in list_top_categories:
                    if category_dict[j]== min_value:
                        list_top_categories[list_top_categories.index(j)]= i
                        break
                        
#     print("printing selected categories in helper function: " + str(list_top_four_categories))                
    for i in list_top_categories:
        dict_top[i]= category_dict[i]
#     print("printing selected categories in helper function: " + str(dict_top_four))
    return dict_top
    
#                 print "category_dict"+ str(category_dict)
#                 print list_top_four_categories

In [281]:
def outputSummary(output_dict, mother_dict):
    output_summary_dict={}
    for i in output_dict:
        count_also_viewed=0
        count_also_bought=0
        count_bought_together=0
        count_buy_after_viewing=0
        asin_count_in_cluster=0

        temp_dict={}
        for j in output_dict[i]['asins']:
            asin_count_in_cluster+=1
            if j in output_dict[i]['also_viewed']:
                count_also_viewed+=1
            if j in output_dict[i]['also_bought']:
                count_also_bought+=1
            if j in output_dict[i]['bought_together']:
                count_bought_together+=1
            if j in output_dict[i]['buy_after_viewing']:
                count_buy_after_viewing+=1

            dict_category_distribution={}
            for k in mother_dict[j]['categories']:
    #             print k
    #             print dict_category_distribution.keys()
                if k not in dict_category_distribution.keys():
                    dict_category_distribution[k]=1
                else:
                    dict_category_distribution[k]+=1   
    #     print "dict_category_distribution for cluster " + str(i)+ ":"+ str(dict_category_distribution)
        top_categories=findTopCategories(dict_category_distribution,3)
    #     print "top_four_categories for cluster "+ str(i)+ ":"+ str(top_four_categories)

#         temp_dict["also_viewed"]= count_also_viewed
#         temp_dict["also_bought"]= count_also_bought
#         temp_dict["bought_together"]= count_bought_together
#         temp_dict["buy_after_viewing"]= count_buy_after_viewing
        temp_dict["asin_count_in_cluster"]= asin_count_in_cluster
        temp_dict["top_categories"]= top_categories
        temp_dict["count_related_products"]= count_also_bought+count_also_viewed+count_bought_together+count_buy_after_viewing

        output_summary_dict[i]=temp_dict
    return output_summary_dict

    #     print ("cluster:" + str(i)+ " "+ "also_viewed"+ str(count_also_viewed)+" "+ "also_bought"+
    #            str(count_also_bought)+" "+"bought_together"+str(count_bought_together)+" "+
    #            "buy_after_viewing"+ str(count_buy_after_viewing)+ " "+ "asin count in cluster"+ str(asin_count_in_cluster))




In [282]:
RAKE_output_summary_dict= outputSummary(RAKE_output_dict, asin_mother_dict)
TFIDF_output_summary_dict= outputSummary(TFIDF_output_dict, asin_mother_dict)

In [283]:
print RAKE_output_summary_dict
print TFIDF_output_summary_dict

{0: {'asin_count_in_cluster': 1, 'count_related_products': 0, 'top_categories': {'Health & Personal Care': 1, 'Health Care': 1, 'Non-Aspirin': 1}}, 1: {'asin_count_in_cluster': 31, 'count_related_products': 4, 'top_categories': {'Health & Personal Care': 1, 'Health Care': 1, "Women's Health": 1}}, 2: {'asin_count_in_cluster': 1, 'count_related_products': 0, 'top_categories': {'Shaving & Hair Removal': 1, 'Health & Personal Care': 1, 'Manual Shaving': 1}}, 3: {'asin_count_in_cluster': 16, 'count_related_products': 0, 'top_categories': {'Health & Personal Care': 1, 'Health Care': 1, 'Sinus Medicine': 1}}, 4: {'asin_count_in_cluster': 12, 'count_related_products': 6, 'top_categories': {'Health & Personal Care': 1, 'Vitamins & Dietary Supplements': 1, 'Vitamin B': 1}}, 5: {'asin_count_in_cluster': 3, 'count_related_products': 0, 'top_categories': {'Health & Personal Care': 1, 'Health Care': 1, 'Absorbent Pads, Shields, Guards & Liners': 1}}, 6: {'asin_count_in_cluster': 3, 'count_related_p